In [13]:
import pandas as pd
pd.set_option('display.max_columns', 1000, 'display.max_colwidth', 1000, 'display.max_rows',1000)

# Read Data

In [14]:
df = pd.read_csv('data/df_raw.csv')
col=[i.replace(' ','') for i in df.columns]
col=[i.replace('=','') for i in col]
col=[str.lower(i) for i in col]
df.columns=col
df.head(1)

,date,btc-usdopen,clfopen,tslaopen,gspcopen,gsptseopen,ixicopen,btc-usddelta,btc-usddirection,clfdelta,clfdirection,tsladelta,tsladirection,gspcdelta,gspcdirection,gsptsedelta,gsptsedirection,ixicdelta,ixicdirection,tweet_date,content,retweetnum,likenum,userhandle,content1,tweet_day,day,score,negative,neutral,positive,combined,sentiment
0,2022-01-26 09:30:00,38421.363281,86.43,952.429993,4408.430176,20781.300781,13868.869141,0.158282,2.0,-0.103958,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-25 16:00:26+00:00,"The numbers are in, and the world still loves Adele—even at her worst https://t.co/RIl3yyFA4V https://t.co/mbI3YbCZrI",91,566,business,the number are in and the world still love adeleeven at her worst,Tuesday,Wednesday,"[0.209, 0.51, 0.281, 0.1027]",0.209,0.51,0.281,0.1027,2


In [15]:
# To clean up texts
import re
import nltk
# nltk.download() Download nltk data for first time use (download all packages)
import nltk.data
# from nltk.stem.snowball import SnowballStemmer
# stemmer = SnowballStemmer('english')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
# tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('omw-1.4')

def sentence_to_wordlist(sentence, remove_stopwords=False):
    # 0. remove mentions(@), Hashtag(#)
    sentence = re.sub(r'@[^\s]+', '', sentence,flags=re.MULTILINE )
    sentence = re.sub(r'#[^\s]+', '', sentence,flags=re.MULTILINE )
    sentence = re.sub(r'RT[^\s]+', '', sentence,flags=re.MULTILINE )
    # 1. drop http
    p1=re.compile(r'http?:\/\/\S+', flags=re.DOTALL)
    sentence = re.sub(p1, '', sentence)
    # 2. drop https
    p1=re.compile(r'https?:\/\/\S+', flags=re.DOTALL)
    sentence = re.sub(p1, '', sentence)
    # 3. Remove non-letters
    sentence = re.sub(r'[^\w\s]','', sentence)
    # 4. Remove all numbers
    sentence = re.sub(r'[0-9]+', '', sentence)
    # 5. Convert words to lower case and split them
    sentence = sentence.lower().split()
    # Remove Stop Words
    sentence = [word for word in sentence if not word in stop_words]
    # 5. Stemming
    # sentence = [stemmer.stem(w) for w in sentence] 
    # 6. Lemmatizing
    sentence = [lemmatizer.lemmatize(word) for word in sentence]

    #check if returned sentence is blank
    if len(sentence)==0:
      return np.nan
    else:
      # 7. Return a sentence of words
      sentence_r = ''
      for word in sentence:
        sentence_r = sentence_r + ' ' + word
      return(sentence_r)

In [16]:
df.content.head(1)

0    The numbers are in, and the world still loves Adele—even at her worst https://t.co/RIl3yyFA4V https://t.co/mbI3YbCZrI
Name: content, dtype: object

In [17]:
df['content2']= df.content.apply(lambda x: sentence_to_wordlist(x))
df['content2'].head(1)

0     number world still love adeleeven worst
Name: content2, dtype: object

In [18]:
col = ['content2','btc-usddirection'] #,'btc-usddirection','tsladirection','gspcdirection','ixicdirection']
df_1 = df[col]
df_1 = df_1.dropna()
X = df_1['content2']
y = df_1[col[1]]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10086, shuffle=False)

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(22868, 20887)

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(22868, 20887)

In [21]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [22]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
])
text_clf = text_clf.fit(X_train, y_train)

In [23]:
import numpy as np
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.4475928112291749

In [24]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                          ('tfidf', TfidfTransformer()),
                          ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                               alpha=0.001,random_state=42)),
                        ])

_ = text_clf_svm.fit(X_train, y_train)
predicted_svm = text_clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)

0.4297520661157025